<a href="https://colab.research.google.com/github/DCDPUAEM/DCDP/blob/main/04%20Deep%20Learning/notebooks/03-Herramientas-Adicionales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Herramientas Adicionales</h1>

El objetivo de esta notebook es mostrar algunas herramientas adicionales para mejorar el entrenamiento y/o desempeño de redes neuronales. En particular, veremos:

* Callbacks
    * Early Stopping
    * Checkpoint
* Dropout
* Gridsearch

Además, realizaremos ejemplos de clasificación binaria.

Recuerda la simbología de las secciones:

* 🔽 Esta sección no forma parte del proceso usual de Machine Learning. Es una exploración didáctica de algún aspecto del funcionamiento del algoritmo.
* ⚡ Esta sección incluye técnicas más avanzadas destinadas a optimizar o profundizar en el uso de los algoritmos.
* ⭕ Esta sección contiene un ejercicio o práctica a realizar. Aún si no se establece una fecha de entrega, es muy recomendable realizarla para practicar conceptos clave de cada tema.

De esta forma podemos verificar que tenemos una GPU:

In [ ]:
import tensorflow as tf

print('GPU presente en: {}'.format(tf.test.gpu_device_name()))

# Callbacks

Un *callback* es un objeto que puede realizar acciones en varias etapas del entrenamiento (por ejemplo, al inicio o al final de una época, antes o después de un *batch*, etc.).

Puedes usar *callbacks* para:

* Escribir los registros de TensorBoard después de cada lote de entrenamiento para monitorizar tus métricas
* Guardar periódicamente tu modelo en el disco
* Hacer un *early stopping*.
* Obtener una visión de los estados internos y las estadísticas de un modelo durante el entrenamiento.

Podemos consultar la lista completa de callbacks en https://keras.io/api/callbacks/

Para ilustrar algunos callbacks, consideremos el siguiente ejemplo ilustrativo. Entrenaremos una red neuronal MLP para la tarea de **clasificación binaria** en el siguiente dataset.

In [ ]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

X, y = make_moons(n_samples=400, noise=0.21, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X,y,train_size=0.8,random_state=199)

plt.figure()
plt.scatter(X_train[:,0],X_train[:,1],c=y_train)
plt.show()

## ⚡ Callbacks: `EarlyStopping`

El [`EarlyStopping`](https://keras.io/api/callbacks/early_stopping/) es un *callback* que nos permite detener el entrenamiento para evitar el overfitting. El callback monitorea el entrenamiento y lo detiene en el momento en el que una métrica (o perdida) deja de mejorar.

Algunos de los principales hiperparámetros son:

* `monitor`: Cantidad a controlar.
* `min_delta`: Cambio mínimo en la cantidad monitorizada para calificar como mejora, es decir, un cambio absoluto inferior a min_delta, contará como no mejora.
* `patience`: Número de épocas sin mejora tras las cuales se detendrá el entrenamiento.
* `mode`: Puede ser {"auto", "min", "max"}. En el modo "min", el entrenamiento se detendrá cuando la cantidad supervisada haya dejado de disminuir; en el modo "max", se detendrá cuando la cantidad supervisada haya dejado de aumentar; en el modo "auto", la dirección se deduce automáticamente del nombre de la cantidad o métrica supervisada.

Definimos y entrenamos una red neuronal. Observa el número de neuronas en la capa de salida y la función de activación.

Su desempeño tendrá un accuracy mucho menor en el conjunto de prueba. Observa las curvas de aprendizaje, **es un caso claro de overfitting**.

*El tiempo de ejecución es de alrededor de 2 minutos*

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

#----- Definimos el modelo -------
model = Sequential()
model.add(Dense(500, input_dim=2, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#----- Entremamos el modelo ------
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=1000, verbose=0)

In [ ]:
# evaluamos el modelo
_, train_acc = model.evaluate(X_train, y_train, verbose=0) # No nos importa guardar el loss en una variable
_, val_acc = model.evaluate(X_val, y_val, verbose=0)
print(f'Train accuracy: {round(train_acc,3)}. Validation accuracy : {round(val_acc,3)}')

# ---- graficamos la función de perdida ----
plt.figure(figsize=(11,5))
plt.subplot(1,2,1)
plt.suptitle("Validation and Training Loss",fontsize=14)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()
# ---- graficamos la métrica de rendimiento ----
plt.subplot(1,2,2)
plt.suptitle("Validation and Training Accuracy",fontsize=14)
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='validation')
plt.legend()
plt.show()

Ahora, usemos el callback para detener el entrenamiento en el momento adecuado

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping


model = Sequential()
model.add(Dense(500, input_dim=2, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Definimos el *callback*. Es una clase por lo que tenemos que inicializarla con hiperparámetros y obtenemos un objeto.

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)

Entrenamos usando el *callback*. Observa en cuántas épocas detiene el entrenamiento.

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                    epochs=1000, verbose=0,
                    callbacks=[es])


_, train_acc = model.evaluate(X_train, y_train, verbose=0)
_, val_acc = model.evaluate(X_val, y_val, verbose=0)
print('Train accuracy: %.3f. Validation accuracy : %.3f' % (train_acc, val_acc))

🔵 Observa que el accuracy en la validación mejoró y en el entrenamiento empeoró un poco, ¿qué interpretación le damos a esto?

Graficamos el entrenamiento

In [ ]:
# ---- graficamos la función de perdida ----
plt.figure(figsize=(11,5))
plt.subplot(1,2,1)
plt.suptitle("Validation and Training Loss",fontsize=14)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()
# ---- graficamos la métrica de rendimiento ----
plt.subplot(1,2,2)
plt.suptitle("Validation and Training Accuracy",fontsize=14)
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='validation')
plt.legend()
plt.show()

## ⚡ Callbacks: `ModelCheckpoint`

Este callback sirve para guardar el modelo en el momento en que comenzó el overfitting y se comenzó a perder accuracy en el conjunto de validación. Algunos parámetros importantes:

* `save_best_only`: if save_best_only=True, it only saves when the model is considered the "best" and the latest best model according to the quantity monitored will not be overwritten. If filepath doesn't contain formatting options like {epoch} then filepath will be overwritten by each new better model.
* `mode`: one of {'auto', 'min', 'max'}. If save_best_only=True, the decision to overwrite the current save file is made based on either the maximization or the minimization of the monitored quantity. For val_acc, this should be max, for val_loss this should be min, etc. In auto mode, the mode is set to max if the quantities monitored are 'acc' or start with 'fmeasure' and are set to min for the rest of the quantities.
* `save_weights_only`: if True, then only the model's weights will be saved (model.save_weights(filepath)), else the full model is saved (model.save(filepath)).

Definimos el modelo

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(500, input_dim=2, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Creamos el callback

In [ ]:
from keras.callbacks import ModelCheckpoint

filepath = 'best_model.hdf5'
checkpoint_best = ModelCheckpoint(filepath=filepath,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='min')
callbacks = [checkpoint_best]

⚡**Por ahora no lo ejecutemos.** También podemos guardar varios modelos, con información sobre la época y loss

In [ ]:
# filepath = 'my_best_model.epoch{epoch:02d}-loss{val_loss:.2f}.hdf5'

# checkpoint_all = ModelCheckpoint(filepath=filepath,
#                              monitor='val_loss',
#                              verbose=1,
#                              save_best_only=True,
#                              mode='min')
# callbacks = [checkpoint_all]

Entrenamos el modelo usando el callback definido previamente. Observar que, en este caso, realizará el entrenamiento con todas las épocas y sólo guardará el módelo cuando alcance un nuevo mínimo en la perdida de la validación.

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100,
                  callbacks=callbacks)

In [ ]:
# ---- graficamos la función de perdida ----
plt.figure(figsize=(11,5))
plt.subplot(1,2,1)
plt.suptitle("Validation and Training Loss",fontsize=14)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()
# ---- graficamos la métrica de rendimiento ----
plt.subplot(1,2,2)
plt.suptitle("Validation and Training Accuracy",fontsize=14)
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='validation')
plt.legend()
plt.show()

### ⚡ Leemos el modelo y realizamos predicciones con él

Leemos y evaluamos usando el modelo guardado del callback anterior.

**En esta parte, además, evaluamos *externamente* las métricas usuales de rendimiento.**

In [ ]:
from keras.models import load_model

model_reloaded = load_model(filepath)
y_pred = model_reloaded.predict(X_val)

In [ ]:
from sklearn.metrics import log_loss

print(f"Binary Cross Entropy: {log_loss(y_val,y_pred)}")

Observa la forma que tienen las predicciones, son probabilidades de pertenecer a la clase positiva (la clase 1). Recuerda que la última capa tiene una activación sigmoide que está en un rango $(0,1)$.

In [ ]:
print(y_pred.shape)
y_pred[:5]

Convirtamoslas a predicciones de clases para fin de evaluar también usando las métricas de rendimiento de clasificación de scikit-learn (precision, accuracy, etc).

In [ ]:
import numpy as np

y_pred_clases = np.where(y_pred>=0.5,1,0).flatten()
print(y_pred_clases.shape)
y_pred_clases[:5]

Ahora sí, podemos evaluar. Recordar que, para el `roc_auc_score` necesitamos las probabilidades de pertenecer a la clase.

In [ ]:
from sklearn.metrics import f1_score, precision_score, roc_auc_score

print(f"Precision Score: {precision_score(y_val,y_pred_clases)}")
print(f"F1 Score: {f1_score(y_val,y_pred_clases)}")
print(f"ROC-AUC Score: {roc_auc_score(y_val,y_pred)}")

También podemos recuperar la entropia binaria cruzada (la función de perdida) a partir del método `evaluate` del modelo.

In [ ]:
evaluation = model_reloaded.evaluate(X_val,y_val)

print(f"Validation loss and validation accuracy: {evaluation}")

# Dropout

Es fácil que las redes neuronales de aprendizaje profundo se sobreajusten rápidamente a un conjunto de datos de entrenamiento con pocos ejemplos.

Se sabe que los conjuntos de redes neuronales con diferentes configuraciones de modelos reducen el sobreajuste, pero requieren el gasto computacional adicional de entrenar y mantener múltiples modelos.

Se puede utilizar un único modelo para simular que se dispone de un gran número de arquitecturas de red diferentes mediante la eliminación aleatoria de nodos durante el entrenamiento. Esto se denomina *dropout* y ofrece un método de regularización muy barato desde el punto de vista computacional y notablemente eficaz para reducir el sobreajuste y mejorar el error de generalización en redes neuronales profundas de todo tipo.

**Esta estrategia no siempre mejora el rendimiento de la red y hay opiniones divididas en cuanto a su eficacia. Sin embargo, es una técnica clásica del deep learning.**


<img align="center" width="50%" src="https://github.com/DCDPUAEM/DCDP/blob/main/04%20Deep%20Learning/img/dropout.png?raw=1"/>

Usaremos el dataset de [diabetes PIMA](https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database) que usamos en el módulo pasado. Usamos este dataset por su tamaño pequeño.

En este ejercicio observaremos como el dropout puede ayudar a prevenir el overfitting, aunque podría no necesariamente mejore la pérdida.

In [ ]:
import pandas as pd
import numpy as np

url = "https://raw.githubusercontent.com/DCDPUAEM/DCDP/main/04%20Deep%20Learning/data/diabetes.csv"

df = pd.read_csv(url,index_col=0)
df

In [ ]:
X = df.iloc[:,:8].values
y = df.iloc[:,8].values

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.875,random_state=89)

print(f"Train size: {X_train.shape[0]}")
print(f"Test size: {X_test.shape[0]}")

Escalaremos los datos para mejorar el rendimiento de la red. Esto, debido a la variedad en los rangos de las variables.

In [ ]:
df.describe()

Realizamos la imputación de valores faltantes

In [ ]:
idxs_to_impute = [1,2,3,4,5]

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=0, strategy='mean')
X_train[:,idxs_to_impute] = imputer.fit_transform(X_train[:,idxs_to_impute])
X_test[:,idxs_to_impute] = imputer.transform(X_test[:,idxs_to_impute])

In [ ]:
df_imputado = pd.DataFrame(X_train)
df_imputado.describe()

Veamos las escalas de valores de cada features

In [ ]:
df_imputado.plot.hist(subplots=True, legend=True)

In [ ]:
fig, axs = plt.subplots(nrows=4, ncols=2,figsize=(16,8))
for col, ax in zip(df_imputado.columns, axs.flatten()):
    df_imputado[col].plot.hist(ax=ax)
fig.show()

Re-escalamos los valores para ponerlos en la misma escala

In [ ]:
from sklearn.preprocessing import StandardScaler

scl = StandardScaler()
X_train = scl.fit_transform(X_train)
X_test = scl.transform(X_test)

Siendo un modelo sencillo, requerimos una arquitectura sencilla. La siguiente es una buena alternativa. No la probaremos por ahora:

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

# model = Sequential()

# model.add(Dense(8, input_dim=8, activation='relu'))
# model.add(Dense(15, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

## Sin dropout

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()

model.add(Dense(8, input_dim=8, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, validation_split=0.1, epochs=50,verbose=0)

In [ ]:
import matplotlib.pyplot as plt

# evaluate the model
_, train_acc = model.evaluate(X_train, y_train, verbose=0)
_, test_acc = model.evaluate(X_test, y_test, verbose=0)
print('Train accuracy: %.3f. Test accuracy : %.3f' % (train_acc, test_acc))

# ---- graficamos la función de perdida ----
plt.figure(figsize=(11,5))
plt.subplot(1,2,1)
plt.suptitle("Validation and Training Loss",fontsize=14)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()
# ---- graficamos la métrica de rendimiento ----
plt.subplot(1,2,2)
plt.suptitle("Validation and Training Accuracy",fontsize=14)
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='validation')
plt.legend()
plt.show()

In [ ]:
model.evaluate(X_test,y_test)

## Usando dropout: Efecto en el overfitting

Usaremos la misma arquitectura general de la red. Añadimos dos capas de dropout, las tasas de dropout fueron seleccionadas con gridsearch.

In [ ]:
from tensorflow.keras.layers import Dropout

def build_model():
    model = Sequential()
    model.add(Dense(8, input_dim=8, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model_do = build_model()

In [ ]:
history = model_do.fit(X_train, y_train, validation_split=0.1, epochs=50,verbose=0)

In [ ]:
import matplotlib.pyplot as plt

# evaluate the model
_, train_acc = model_do.evaluate(X_train, y_train, verbose=0)
_, test_acc = model_do.evaluate(X_test, y_test, verbose=0)
print('Train accuracy: %.3f. Test accuracy : %.3f' % (train_acc, test_acc))

# ---- graficamos la función de perdida ----
plt.figure(figsize=(11,5))
plt.subplot(1,2,1)
plt.suptitle("Validation and Training Loss",fontsize=14)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()
# ---- graficamos la métrica de rendimiento ----
plt.subplot(1,2,2)
plt.suptitle("Validation and Training Accuracy",fontsize=14)
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='validation')
plt.legend()
plt.show()

In [ ]:
model_do.evaluate(X_test,y_test)

Aumentemos el número de épocas, **observa el efecto del hiperparámetro `patience`**

In [ ]:
from keras.callbacks import EarlyStopping

model_do_2 = build_model()

es = EarlyStopping(patience=3)

history = model_do_2.fit(X_train, y_train, validation_split=0.1, epochs=200,verbose=0,
                       callbacks=[es])

In [ ]:
# ---- evaluamos el modelo ----
_, train_acc = model_do_2.evaluate(X_train, y_train, verbose=0)
_, test_acc = model_do_2.evaluate(X_test, y_test, verbose=0)
print('Train accuracy: %.3f. Test accuracy : %.3f' % (train_acc, test_acc))

# ---- graficamos la función de perdida ----
plt.figure(figsize=(11,5))
plt.subplot(1,2,1)
plt.suptitle("Validation and Training Loss",fontsize=14)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()
# ---- graficamos la métrica de rendimiento ----
plt.subplot(1,2,2)
plt.suptitle("Validation and Training Accuracy",fontsize=14)
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='validation')
plt.legend()
plt.show()

# ⚡⚡ Gridsearch

A continuación se muestra cómo realizar un gridsearch para obtener los mejores parámetros de una red neuronal, es decir, los que producen las mejores métricas. Estos parámetros pueden ser el número de neuronas, la tasa de dropout, las épocas, etc.

Para poder usar el gridsearch de scikit-learn es necesario *traducir* el módelo de red neuronal a un clasificador de scikit-learn. Esto lo hacemos con la clase `KerasClassifier`.

**⚠❗Warning**: Si se especifican un gran número de parámetros en la busqueda, esta puede tardar mucho y pueden ser penalizados en el uso de GPU en Colab. Usar con cuidado.

Seguimos con el dataset de [diabetes](https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database). Resumamos el preprocesamiento en una sola celda:

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

url = "https://raw.githubusercontent.com/DCDPUAEM/DCDP/main/04%20Deep%20Learning/data/diabetes.csv"

df = pd.read_csv(url,index_col=0)

X = df.iloc[:,:8].values
y = df.iloc[:,8].values

X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.875,random_state=89)

idxs_to_impute = [1,2,3,4,5]
imputer = SimpleImputer(missing_values=0, strategy='mean')
X_train[:,idxs_to_impute] = imputer.fit_transform(X_train[:,idxs_to_impute])
X_test[:,idxs_to_impute] = imputer.transform(X_test[:,idxs_to_impute])

scl = StandardScaler()
X_train = scl.fit_transform(X_train)
X_test = scl.transform(X_test)

Es necesario crear una función que cree el modelo, esta debe depender de los parámetros sobre los que se quiere realizar la busqueda. Es necesario crear el modelo, compilarlo y regresarlo ya compilado.

Con la finalidad de no usar muchos recursos, problemas con un modelo muy sencillo, en el cual variaremos:

* El número de neuronas en la capa oculta.
* La activación de la capa oculta.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

def create_model(n_neurons,activation):
	model = Sequential()
	model.add(Dense(n_neurons, input_dim=8, activation=activation))
	model.add(Dense(1, activation='sigmoid'))
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

Creamos un modelo de clasificador de scikit-learn usando la API de Keras. Esta empaqueta el módelo de keras como un estimador de scikit-learn.

[Documentación](https://adriangb.com/scikeras/stable/quickstart.html#training-a-model)

In [ ]:
!pip install scikeras[tensorflow]

In [ ]:
from scikeras.wrappers import KerasClassifier
from keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss',patience=2)

model = KerasClassifier(
    create_model,
    n_neurons=12,
    activation='sigmoid',
    epochs=50,
    verbose=0,
    callbacks=[es],
    validation_split=0.15
)

Realizamos la busqueda de parámetros

In [ ]:
from sklearn.model_selection import GridSearchCV

# ----- Definimos los parámetros de la busqueda -----
neurons = [10,15,20,30]
activations = ['relu','sigmoid','tanh']
param_grid = {'n_neurons':neurons,
              'activation':activations}

# ----- Definimos y realizamos el gridsearch
gs = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = gs.fit(X_train, y_train)

Veamos los mejores parámetros

In [ ]:
print(f"Best Accuracy: {grid_result.best_score_} using parameters {grid_result.best_params_}")

Podemos acceder al mejor modelo y usarlo.

**Observación**:  El modelo, aún cuando es una red neuronal, está presentado como un estimador de scikit-learn por lo que no tiene método `evaluate` (este es de keras), sino `score` (este es de sklearn).

In [ ]:
best_model = gs.best_estimator_

best_model.score(X_test,y_test)

# ⭕ Práctica

En esta práctica haremos regresión multi-output usando el dataset `mo_regression.csv` de la carpeta data del módulo en el repositorio.

0. Divide el conjunto de datos en 80% entrenamiento y 20% prueba.
1. Define dos modelos de red neuronal MLP para resolver esta tarea. La función de perdida a usar será 'MSE'. Un módelo tendrá una arquitectura *sencilla* y el otro, una arquitectura *compleja*.
2. Prueba diferentes combinaciones de hiperparámetros para encontrar el mejor modelo que puedas en cuanto a desempeño en el conjunto de prueba, usando la métrica MAE.

In [ ]:
import pandas as pd
import numpy as np

url = "https://raw.githubusercontent.com/DCDPUAEM/DCDP/main/04%20Deep%20Learning/data/mo_regression.csv"

df = pd.read_csv(url,index_col=0)
df